In [1]:
import feedparser
from pprint import pprint
from bs4 import BeautifulSoup
import pandas as pd

url = "http://news.google.com/news?q=covid&hl=en-US&sort=date&gl=US&num=10&output=rss"

class ParseFeed():
    def __init__(self, url):
        self.feed_url = url
        
    def clean(self, html):
        '''
        Get the text from html and do some cleaning
        '''
        soup = BeautifulSoup(html)
        text = soup.get_text()
        text = text.replace('\xa0', ' ')
        return text

    def parse(self):
        newsDescription = []
        newsUrl = []
        newsDate = []
        
        '''
        Parse the URL, and print all the details of the news 
        '''        
        feeds = feedparser.parse(self.feed_url).entries
        
        for f in feeds:
            newsDescription.append(self.clean(f.get("description", "")))
            newsUrl.append(f.get("link", ""))
            newsDate.append(f.get("published", ""))
            
        df = pd.DataFrame({'Description':newsDescription, 'URL':newsUrl, 'Date Published':newsDate})            
        return df
    
htmlOutput = ""
feed = ParseFeed(url)
df = feed.parse()

In [2]:
df['Date'] = pd.to_datetime(df['Date Published'])
df = df.head(10).sort_values(by = ['Date'], ascending=False)

# Saving data to html output
htmlOutput = """<html><title>COVID-19 Daily News</title><body><style>
		body {
		  background-image: url('background.jpg');
		  background-repeat: no-repeat;
		  background-attachment: fixed;
		  background-size: cover;
		}
	  </style><center><h1>COVID-19 Daily News</h1></center><dl>"""

for index,row in df.iterrows():
    htmlOutput = htmlOutput + "<dt>" + row['Description'] + "</dt><dd>" + row['Date Published'] + "</dd><dd><a href =" + row['URL'] + ">" + row['URL'] + "</a></dd><br>"
htmlOutput = htmlOutput + "</dl></body></html>"
text_file = open("home.html", "w")
text_file.write(htmlOutput)
text_file.close()

df.to_csv('data.csv', index=False)
df

,Description,URL,Date Published,Date
7,Pennsylvania is limiting who administers the C...,https://www.inquirer.com/news/covid-vaccine-pe...,"Sun, 14 Feb 2021 02:48:45 GMT",2021-02-14 02:48:45+00:00
5,New Zealand reports three new COVID-19 local c...,https://www.reuters.com/article/us-health-coro...,"Sun, 14 Feb 2021 01:46:00 GMT",2021-02-14 01:46:00+00:00
3,Health officials report 43 more COVID-19 death...,https://www.kgw.com/article/news/health/corona...,"Sun, 14 Feb 2021 00:04:00 GMT",2021-02-14 00:04:00+00:00
9,No changes for Flyers on COVID-19 list The Ph...,https://www.inquirer.com/flyers/philadelphia-f...,"Sat, 13 Feb 2021 23:26:15 GMT",2021-02-13 23:26:15+00:00
2,10 more dead from COVID-19 in Wisconsin; 71 ne...,https://wkow.com/2021/02/13/10-more-dead-from-...,"Sat, 13 Feb 2021 20:32:32 GMT",2021-02-13 20:32:32+00:00
1,Governor Cuomo Announces COVID-19 7-Day Averag...,https://www.governor.ny.gov/news/governor-cuom...,"Sat, 13 Feb 2021 16:05:43 GMT",2021-02-13 16:05:43+00:00
8,Feb. 13 update on COVID-19 in MN: Vaccination ...,https://www.mprnews.org/story/2021/02/13/lates...,"Sat, 13 Feb 2021 13:00:00 GMT",2021-02-13 13:00:00+00:00
4,Covid: Claims vaccinations harm fertility unfo...,https://www.bbc.com/news/health-56012529,"Sat, 13 Feb 2021 02:01:00 GMT",2021-02-13 02:01:00+00:00
0,FDA.gov Coronavirus (COVID-19) Update: Februar...,https://www.fda.gov/news-events/press-announce...,"Fri, 12 Feb 2021 22:12:00 GMT",2021-02-12 22:12:00+00:00
6,US Department of Education COVID-19 Handbook V...,https://www.ed.gov/news/press-releases/us-depa...,"Fri, 12 Feb 2021 19:16:13 GMT",2021-02-12 19:16:13+00:00
